In [100]:
import pandas as pd

#### DDA 선택 기준
- 분석 제공 대상 선정: 컬럼 선택 기준 달라짐. 
    - 환자 
    - 의사 
    - 병원 운영 관리자 
    - 보호자 

In [101]:
df_ROS = pd.read_csv('../../../datasets/RecurrenceOfSurgery.csv')
df_ROS.head(2)

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,0,1PT,22.8,3,51.0,0.0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
1,1,2PT,44.9,4,26.0,0.0,0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.1,4,Up,0


In [102]:
df_ROS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 52 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              1894 non-null   int64  
 1   환자ID                    1894 non-null   object 
 2   Large Lymphocyte        1894 non-null   float64
 3   Location of herniation  1894 non-null   int64  
 4   ODI                     462 non-null    float64
 5   가족력                     1843 non-null   float64
 6   간질성폐질환                  1894 non-null   int64  
 7   고혈압여부                   1894 non-null   int64  
 8   과거수술횟수                  1894 non-null   int64  
 9   당뇨여부                    1894 non-null   int64  
 10  말초동맥질환여부                1894 non-null   int64  
 11  빈혈여부                    1894 non-null   int64  
 12  성별                      1894 non-null   int64  
 13  스테로이드치료                 1894 non-null   int64  
 14  신부전여부                   1894 non-null   

### 데이터 핸들링 

- 정렬 (sort할때 어떤 걸 기준으로 앞으로 세울지, 뒤에 세울지)
 df.sort_values(by=[''], ascending=[])
- 추출 (top 몇개) 
    - 열 단위 추출: df[['', '']] 컬럼 리스트로 한다 
    - 행 단위 추출: iloc[:3]  0번째 행 ~ 2번째 행
- 필터 (특정값을 뽑아냄): df.loc[conditions]
- 요약: 피봇틀, 원하는형태로 변환시킬 수 있다 (통계 낼 수 있는 기능있는 function)
pivot_table(index = , values=, aggfunc=)
-index: group by
-values: 대상 컬럼명
-aggfunc: 집합 함수(from numpy)

- --- + 전처리 

### 어느대상에게 얘기할지 데이터 보고 판단 :
### 환자 대상 분석 필요
0. 변수 구조 확인 
1. 목표 변수 설정 : '재발여부'로 정의
2. 설명 변수 설정 : '성별', '신장', '심혈관질환', '체중', '흡연여부', '연령', '직업', '혈액형' (연관되어있는 것 모으기) 

In [103]:
df_ROS.columns

Index(['Unnamed: 0', '환자ID', 'Large Lymphocyte', 'Location of herniation',
       'ODI', '가족력', '간질성폐질환', '고혈압여부', '과거수술횟수', '당뇨여부', '말초동맥질환여부', '빈혈여부',
       '성별', '스테로이드치료', '신부전여부', '신장', '심혈관질환', '암발병여부', '연령', '우울증여부', '입원기간',
       '입원일자', '종양진행여부', '직업', '체중', '퇴원일자', '헤모글로빈수치', '혈전합병증여부', '환자통증정도',
       '흡연여부', '통증기간(월)', '수술기법', '수술시간', '수술실패여부', '수술일자', '재발여부', '혈액형',
       '전방디스크높이(mm)', '후방디스크높이(mm)', '지방축적도', 'Instability', 'MF + ES',
       'Modic change', 'PI', 'PT', 'Seg Angle(raw)', 'Vaccum disc', '골밀도',
       '디스크단면적', '디스크위치', '척추이동척도', '척추전방위증'],
      dtype='object')

In [60]:
# '성별', '신장', '심혈관질환', '체중', '흡연여부', '연령', '직업', '혈액형', '재발여부'
# 열 단위 추출  (시리즈로 출력)
# df_ROS['재발여부']
# df_ROS[['재발여부']] (컬럼 리스트)
# df_ROS[['성별', '신장', '심혈관질환', '체중', '흡연여부', '연령', '직업', '혈액형', '재발여부']]
df_ROS_forpatient = df_ROS[['성별', '신장', '심혈관질환', '체중', '흡연여부', '연령', '직업', '혈액형', '재발여부']]
# df_ROS_forpatient.head(2)
df_ROS_forpatient.tail(3)

,성별,신장,심혈관질환,체중,흡연여부,연령,직업,혈액형,재발여부
1891,1,167,0,70.0,0,61,무직,RH+O,0
1892,1,177,0,77.0,0,29,사무직,RH+A,0
1893,2,168,0,49.0,0,30,사무직,RH+AB,0


In [104]:
# 행단위 추출 
# iloc = index location (인덱스 번호 이용하는 방법, 데이터 순서가 바뀔 때도 있다)
# iloc[:, :], head(), tail()
#[:3] : 0번째 행 ~ 2번쨰 행 
df_ROS_forpatient.iloc[:3] 

,성별,신장,심혈관질환,체중,흡연여부,연령,직업,혈액형,재발여부
0,2,163,0,60.3,0,66,자영업,RH+A,0
1,1,171,0,71.7,0,47,운동선수,RH+A,0
2,1,178,0,77.1,0,39,특수전문직,RH+B,0


In [105]:
#[-4:] 뒤에서부터 4번째 행부터 끝까지 
df_ROS_forpatient.iloc[-4:]

,성별,신장,심혈관질환,체중,흡연여부,연령,직업,혈액형,재발여부
1890,2,157,0,59.0,0,42,사무직,RH+B,0
1891,1,167,0,70.0,0,61,무직,RH+O,0
1892,1,177,0,77.0,0,29,사무직,RH+A,0
1893,2,168,0,49.0,0,30,사무직,RH+AB,0


### index 관련 사항 
### 시각화할때 편의성 위해 index를 콜룸으로 이름 만든다 (index 셋팅) 


In [63]:
df_ROS.set_index('환자ID')
#유니크한 컬럼 

,Unnamed: 0,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,말초동맥질환여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
환자ID,,,,,,,,,,,,,,,,,,,,,
1PT,0,22.8,3,51.0,0.0,0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.50,4,Down,0
2PT,1,44.9,4,26.0,0.0,0,0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.10,4,Up,0
3PT,2,53.0,3,NaN,0.0,0,0,1,0,0,...,0,67.5,27.3,10.2,0,-1.77,1749.00,4,Down,0
4PT,3,54.7,4,18.0,0.0,0,1,0,0,0,...,0,49.2,18.7,19.9,0,-1.58,1796.20,4,Down,0
5PT,4,53.8,4,NaN,0.0,0,1,0,0,0,...,0,58.8,14.7,5.2,0,-0.79,2021.48,4,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890PT,1889,44.0,1,NaN,0.0,0,0,0,0,0,...,2,59.5,23.0,21.8,0,NaN,2652.20,5,Middle,0
1891PT,1890,30.8,2,NaN,0.0,0,0,0,0,0,...,0,47.7,20.2,5.0,0,NaN,1795.80,4,Down,0
1892PT,1891,36.7,3,NaN,0.0,0,0,0,0,1,...,0,44.6,15.0,17.4,1,NaN,2624.40,4,Down,0


In [64]:
df_ROS = df_ROS.set_index('성별')
# 유니크하지 않은 컬럼 

In [106]:
df_ROS.iloc[-4:]

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
1890,1890,1891PT,30.8,2,NaN,0.0,0,0,0,0,...,0,47.7,20.2,5.0,0,NaN,1795.8,4,Down,0
1891,1891,1892PT,36.7,3,NaN,0.0,0,0,0,0,...,0,44.6,15.0,17.4,1,NaN,2624.4,4,Down,0
1892,1892,1893PT,32.5,2,NaN,0.0,0,0,0,0,...,2,32.2,11.1,17.7,0,NaN,1972.5,5,Down,0
1893,1893,1894PT,49.5,2,NaN,0.0,0,0,0,0,...,0,38.9,6.8,27.8,0,NaN,1574.4,5,Middle,0


In [66]:
### 정렬: 어떤 콜룸으로 정렬? des/asc 중에 어떤거? (기본적으로 asc오름차순)  항목 여러개 넣을때는 리스트 형식으로 
df_ROS_forpatient.sort_values(by=['신장'])

,성별,신장,심혈관질환,체중,흡연여부,연령,직업,혈액형,재발여부
1763,2,140,1,35.0,0,86,무직,RH+O,0
896,2,140,0,35.0,0,48,NaN,RH+A,0
348,2,142,0,52.9,0,38,자영업,RH+A,0
70,2,143,0,53.3,0,31,사무직,RH+O,0
974,2,143,0,56.0,0,56,무직,RH+O,0
...,...,...,...,...,...,...,...,...,...
202,1,189,0,85.3,0,22,군인,RH+B,0
464,1,189,0,85.9,0,39,자영업,RH+A,0
1227,1,190,0,74.0,0,47,자영업,RH+O,0
1246,1,191,0,114.0,0,17,학생,RH+O,0


In [67]:
df_ROS_forpatient.sort_values(by=['신장'], ascending=[False])
#내림차순 ascending=[False] 오름차순 ascending=[True]

,성별,신장,심혈관질환,체중,흡연여부,연령,직업,혈액형,재발여부
1579,1,204,0,103.0,0,39,교사,RH+O,1
1246,1,191,0,114.0,0,17,학생,RH+O,0
1227,1,190,0,74.0,0,47,자영업,RH+O,0
464,1,189,0,85.9,0,39,자영업,RH+A,0
202,1,189,0,85.3,0,22,군인,RH+B,0
...,...,...,...,...,...,...,...,...,...
70,2,143,0,53.3,0,31,사무직,RH+O,0
974,2,143,0,56.0,0,56,무직,RH+O,0
348,2,142,0,52.9,0,38,자영업,RH+A,0
1763,2,140,1,35.0,0,86,무직,RH+O,0


In [68]:
df_ROS_forpatient.sort_values(by=['신장', '체중'], ascending=[False, False])
#신장이 먼저 정렬되고 그 안의 범위에서 체중 정렬 

,성별,신장,심혈관질환,체중,흡연여부,연령,직업,혈액형,재발여부
1579,1,204,0,103.0,0,39,교사,RH+O,1
1246,1,191,0,114.0,0,17,학생,RH+O,0
1227,1,190,0,74.0,0,47,자영업,RH+O,0
464,1,189,0,85.9,0,39,자영업,RH+A,0
202,1,189,0,85.3,0,22,군인,RH+B,0
...,...,...,...,...,...,...,...,...,...
974,2,143,0,56.0,0,56,무직,RH+O,0
70,2,143,0,53.3,0,31,사무직,RH+O,0
348,2,142,0,52.9,0,38,자영업,RH+A,0
896,2,140,0,35.0,0,48,NaN,RH+A,0


In [78]:
## quest 
## 정렬: 혈액형(ascending), 직업(discending) 
df_ROS_list = df_ROS.sort_values(by=['혈액형', '직업'], ascending=[True, False])
df_ROS_list

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
60,60,61PT,48.2,2,15.0,0.0,0,0,0,0,...,0,42.5,11.4,12.8,0,-2.18,2291.70,4,Down,0
176,176,177PT,53.1,2,NaN,0.0,0,0,0,0,...,0,37.3,2.9,22.5,0,-1.75,1850.78,4,Middle,0
184,184,185PT,46.9,1,NaN,0.0,0,0,0,0,...,0,54.4,16.1,26.2,0,-0.72,2011.62,4,Middle,0
224,224,225PT,56.5,1,NaN,0.0,0,0,0,0,...,0,50.8,13.0,14.5,0,-1.04,1624.75,3,Down,0
272,272,273PT,50.2,1,NaN,1.0,0,0,0,0,...,0,45.4,8.4,15.4,0,-1.54,1973.00,4,Middle,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1710,1710,1711PT,44.7,2,NaN,NaN,0,0,0,0,...,0,55.8,22.6,32.3,0,NaN,2141.75,4,Down,0
1715,1715,1716PT,50.9,3,NaN,NaN,0,0,0,0,...,0,52.4,15.5,15.5,0,NaN,1931.25,5,Middle,0
1718,1718,1719PT,36.7,1,NaN,NaN,0,0,0,0,...,0,39.9,11.7,16.8,0,NaN,1582.24,5,Up,0
1726,1726,1727PT,57.5,2,NaN,NaN,0,0,0,0,...,0,45.4,4.9,25.6,0,NaN,2404.30,4,Up,0


In [81]:
## 추출: 혈액형, 직업, 성별 --> 표시 
df_ROS_forpatient_2 = df_ROS_list[['혈액형', '직업', '성별']]
df_ROS_forpatient_2.iloc[:5]

,혈액형,직업,성별
60,RH+A,학생,1
176,RH+A,학생,1
184,RH+A,학생,2
224,RH+A,학생,2
272,RH+A,학생,1


In [ ]:
## 
- 직업군이 상위 2위에 속하는 항목: value_count, Dataframe, iloc
- 체중 > 70, 신장 > 170, 흡연여부 없는 분 
- 해당 레코드 갯수 표현 

In [134]:
df_ROS_JOB = df_ROS['직업'].value_counts()

In [136]:
df_ROS_JOB.iloc[:2]

사무직    449
자영업    171
Name: 직업, dtype: int64

In [141]:
condition = (df_ROS['직업'] == '자영업') | (df_ROS['직업'] == '사무직') 
df_ROS_condition1 = df_ROS.loc[condition]
df_ROS_condition1

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,0,1PT,22.8,3,51.0,0.0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
5,5,6PT,28.5,3,NaN,0.0,0,0,2,0,...,0,41.8,23.6,8.8,0,-0.95,1937.5,4,Down,0
11,11,12PT,53.2,3,39.0,0.0,0,0,0,0,...,2,37.2,8.9,13.5,1,-1.35,2119.3,4,Middle,0
23,23,24PT,28.5,3,NaN,0.0,0,1,0,0,...,0,41.8,20.8,10.3,1,-1.69,2205.2,4,Down,0
30,30,31PT,51.1,3,NaN,0.0,0,0,0,0,...,0,45.3,14.9,20.2,0,-1.64,1833.2,4,Middle,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1883,1883,1884PT,25.4,3,NaN,0.0,0,1,0,0,...,0,55.2,25.1,11.5,0,NaN,2381.2,4,Down,0
1888,1888,1889PT,42.4,3,NaN,0.0,0,0,0,0,...,0,54.2,21.3,6.8,1,NaN,2421.3,3,Down,0
1890,1890,1891PT,30.8,2,NaN,0.0,0,0,0,0,...,0,47.7,20.2,5.0,0,NaN,1795.8,4,Down,0
1892,1892,1893PT,32.5,2,NaN,0.0,0,0,0,0,...,2,32.2,11.1,17.7,0,NaN,1972.5,5,Down,0


In [142]:
condition2 = (df_ROS_condition1['체중'] > 70 ) & (df_ROS_condition1['신장'] > 170) & (df_ROS_condition1['흡연여부'] == 0)
df_ROS_condition2 = df_ROS_condition1.loc[condition2]
df_ROS_condition2

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
23,23,24PT,28.5,3,NaN,0.0,0,1,0,0,...,0,41.8,20.8,10.3,1,-1.69,2205.2,4,Down,0
30,30,31PT,51.1,3,NaN,0.0,0,0,0,0,...,0,45.3,14.9,20.2,0,-1.64,1833.2,4,Middle,0
40,40,41PT,42.7,5,NaN,0.0,0,0,0,0,...,2,35.8,16.1,14.6,0,-1.98,2040.7,4,Middle,0
53,53,54PT,38.6,3,NaN,0.0,0,0,0,0,...,0,33.6,10.6,7.0,0,-1.10,2099.3,4,Middle,0
72,72,73PT,34.6,2,NaN,0.0,0,0,0,0,...,0,38.4,16.2,16.1,0,-2.34,2387.1,3,Down,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1830,1830,1831PT,47.9,1,NaN,0.0,0,0,0,0,...,0,56.3,19.7,26.4,0,NaN,2310.0,4,Middle,0
1841,1841,1842PT,33.1,1,NaN,0.0,0,0,0,0,...,0,39.2,14.3,9.1,0,NaN,2067.0,4,Middle,0
1855,1855,1856PT,55.6,2,NaN,0.0,0,0,0,0,...,0,58.8,15.8,18.6,0,NaN,2149.3,4,Middle,0
1871,1871,1872PT,51.1,1,NaN,0.0,0,0,0,0,...,0,42.1,7.0,24.9,0,NaN,2102.3,4,Extremely down,0


In [143]:
len(df_ROS_condition2) 

164

In [148]:
list_df_ROS = list(df_ROS['직업'].value_counts().iloc[:2].index)

In [155]:
df_ROS['직업'].value_counts().iloc[:2].values

array([449, 171], dtype=int64)

In [156]:
# df_ROS['직업'].value_counts().index[0]
list_df_ROS[0]

'사무직'

In [157]:
# df_ROS['직업'].value_counts().index[1]
list_df_ROS[1]

'자영업'

In [158]:
condition = (df_ROS['직업'] == list_df_ROS[1]) | (df_ROS['직업'] == list_df_ROS[0]) 
df_ROS_condition1 = df_ROS.loc[condition]
df_ROS_condition1

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,0,1PT,22.8,3,51.0,0.0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
5,5,6PT,28.5,3,NaN,0.0,0,0,2,0,...,0,41.8,23.6,8.8,0,-0.95,1937.5,4,Down,0
11,11,12PT,53.2,3,39.0,0.0,0,0,0,0,...,2,37.2,8.9,13.5,1,-1.35,2119.3,4,Middle,0
23,23,24PT,28.5,3,NaN,0.0,0,1,0,0,...,0,41.8,20.8,10.3,1,-1.69,2205.2,4,Down,0
30,30,31PT,51.1,3,NaN,0.0,0,0,0,0,...,0,45.3,14.9,20.2,0,-1.64,1833.2,4,Middle,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1883,1883,1884PT,25.4,3,NaN,0.0,0,1,0,0,...,0,55.2,25.1,11.5,0,NaN,2381.2,4,Down,0
1888,1888,1889PT,42.4,3,NaN,0.0,0,0,0,0,...,0,54.2,21.3,6.8,1,NaN,2421.3,3,Down,0
1890,1890,1891PT,30.8,2,NaN,0.0,0,0,0,0,...,0,47.7,20.2,5.0,0,NaN,1795.8,4,Down,0
1892,1892,1893PT,32.5,2,NaN,0.0,0,0,0,0,...,2,32.2,11.1,17.7,0,NaN,1972.5,5,Down,0


In [165]:
condition2 = (df_ROS_condition1['체중'] > 70 ) & (df_ROS_condition1['신장'] > 170) & (df_ROS_condition1['흡연여부'] == 0)
df_ROS_condition2 = df_ROS_condition1.loc[condition2]
df_ROS_condition2

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
23,23,24PT,28.5,3,NaN,0.0,0,1,0,0,...,0,41.8,20.8,10.3,1,-1.69,2205.2,4,Down,0
30,30,31PT,51.1,3,NaN,0.0,0,0,0,0,...,0,45.3,14.9,20.2,0,-1.64,1833.2,4,Middle,0
40,40,41PT,42.7,5,NaN,0.0,0,0,0,0,...,2,35.8,16.1,14.6,0,-1.98,2040.7,4,Middle,0
53,53,54PT,38.6,3,NaN,0.0,0,0,0,0,...,0,33.6,10.6,7.0,0,-1.10,2099.3,4,Middle,0
72,72,73PT,34.6,2,NaN,0.0,0,0,0,0,...,0,38.4,16.2,16.1,0,-2.34,2387.1,3,Down,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1830,1830,1831PT,47.9,1,NaN,0.0,0,0,0,0,...,0,56.3,19.7,26.4,0,NaN,2310.0,4,Middle,0
1841,1841,1842PT,33.1,1,NaN,0.0,0,0,0,0,...,0,39.2,14.3,9.1,0,NaN,2067.0,4,Middle,0
1855,1855,1856PT,55.6,2,NaN,0.0,0,0,0,0,...,0,58.8,15.8,18.6,0,NaN,2149.3,4,Middle,0
1871,1871,1872PT,51.1,1,NaN,0.0,0,0,0,0,...,0,42.1,7.0,24.9,0,NaN,2102.3,4,Extremely down,0


In [160]:
len(df_ROS_condition2) 

164

# Quest 3 
- quest 2 최종 데이터 사용  
- 직업별 성별 빈도
- 직업별 혈액형 빈도 

In [204]:
import numpy as np
df_ROS_condition2.pivot_table(index = '직업', columns ='성별', aggfunc = 'size')

성별,1,2
직업,,
사무직,121.0,4.0
자영업,39.0,NaN


In [194]:
df_ROS_condition2.pivot_table(index = '직업', columns = '혈액형', aggfunc = 'size'  )

혈액형,RH+A,RH+AB,RH+B,RH+O
직업,,,,
사무직,50,13,24,38
자영업,21,3,8,7


In [201]:
df_ROS_condition2[['성별', '직업']].pivot_table(index = '직업', columns ='성별', aggfunc = 'size')

성별,1,2
직업,,
사무직,121.0,4.0
자영업,39.0,NaN


In [206]:
df_ROS_condition2[['혈액형', '직업']].pivot_table(index = '직업', columns ='혈액형', aggfunc = 'size')

혈액형,RH+A,RH+AB,RH+B,RH+O
직업,,,,
사무직,50,13,24,38
자영업,21,3,8,7
